In [20]:
import tensorflow as tf 
import numpy as np
import pandas as pd

In [21]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU') 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
y_train.shape

(50000, 1)

In [8]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [9]:
scaled_y_train = tf.keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
scaled_y_test = tf.keras.utils.to_categorical(y_test, num_classes=10, dtype='float32' )

In [10]:
scaled_y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [11]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (32, 32, 3)),
        tf.keras.layers.Dense(3000, activation="relu"), 
        tf.keras.layers.Dense(1000, activation="relu"), 
        tf.keras.layers.Dense(10, activation="relu"), 
    ])

    model.compile(
        optimizer="SGD",
        loss= "categorical_crossentropy",
        metrics= ["accuracy"]
    )

    return model

In [12]:
scaled_X_train = X_train/255
scaled_X_test = X_test/255

In [13]:

scaled_X_train = np.array(scaled_X_train)
scaled_y_train = np.array(scaled_y_train)

scaled_X_test = np.array(scaled_X_test)
scaled_y_test = np.array(scaled_y_test)

tensor_train_dataset = tf.data.Dataset.from_tensor_slices((scaled_X_train, scaled_y_train))
tensor_test_dataset = tf.data.Dataset.from_tensor_slices((scaled_X_test, scaled_y_test))

In [14]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [15]:
strategy.num_replicas_in_sync

1

In [16]:
#considering 4 replicas

batch_size_per_replica =250
BATCH_SIZE =  batch_size_per_replica * strategy.num_replicas_in_sync

train_data = tensor_train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [17]:
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_data, epochs=10)

Epoch 1/10
200/200 [==============================] - 10s 48ms/step - loss: 2.4068 - accuracy: 0.1000
Epoch 2/10
200/200 [==============================] - 10s 48ms/step - loss: 2.2970 - accuracy: 0.1046
Epoch 3/10
200/200 [==============================] - 10s 48ms/step - loss: 2.2856 - accuracy: 0.1806
Epoch 4/10
200/200 [==============================] - 10s 49ms/step - loss: 2.2763 - accuracy: 0.2255
Epoch 5/10
200/200 [==============================] - 10s 49ms/step - loss: 2.2605 - accuracy: 0.2383
Epoch 6/10
200/200 [==============================] - 10s 49ms/step - loss: 2.2215 - accuracy: 0.2483
Epoch 7/10
200/200 [==============================] - 10s 51ms/step - loss: 2.1627 - accuracy: 0.2317
Epoch 8/10
200/200 [==============================] - 11s 54ms/step - loss: 2.2379 - accuracy: 0.1479
Epoch 9/10
200/200 [==============================] - 11s 54ms/step - loss: 2.1417 - accuracy: 0.2119
Epoch 10/10
200/200 [==============================] - 11s 53ms/step - loss: 2.226

RuntimeError: Physical devices cannot be modified after being initialized